Use python version 3.10.15

# Setup

## Import Dependencies

In [1]:
import numpy as np
import timeit
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format
from utils import _generate_random_matrices, block_split, algorithm_from_factors, generate_naive_factorization, _get_n_from_factors, _get_2x2x2_strassen, pad, _generate_random_int_matrices

## Load Files

In [2]:
standard_factorizations_path = "/Users/kev/Documents/projects/alphatensor-analysis/algorithms/factorizations_r.npz"
mod2_factorizations_path = "/Users/kev/Documents/projects/alphatensor-analysis/algorithms/factorizations_f2.npz"

standard_factorizations = dict(np.load(standard_factorizations_path, allow_pickle=True))
mod2_factorizations = dict(np.load(mod2_factorizations_path, allow_pickle=True))

## Print Ranks

In [6]:
# Print available factorizations and their shapes.
for key in standard_factorizations:
  naive_u, v, w = standard_factorizations[key]
  rank = naive_u.shape[-1]
  assert rank == v.shape[-1] and rank == w.shape[-1]
  print(f'{key}: rank={naive_u.shape[-1]}')

2,2,2: rank=7
2,2,3: rank=11
2,2,4: rank=14
2,2,5: rank=18
2,2,6: rank=21
2,2,7: rank=25
2,2,8: rank=28
2,3,3: rank=15
2,3,4: rank=20
2,3,5: rank=25
2,4,4: rank=26
2,4,5: rank=33
2,5,5: rank=40
3,3,3: rank=23
3,3,4: rank=29
3,3,5: rank=36
3,4,4: rank=38
3,4,5: rank=47
3,4,11: rank=103
3,5,5: rank=58
3,5,9: rank=105
3,9,11: rank=225
4,4,4: rank=49
4,4,5: rank=63
4,5,5: rank=76
4,5,9: rank=139
4,5,10: rank=152
4,5,11: rank=169
4,9,10: rank=255
4,9,11: rank=280
4,11,11: rank=343
4,11,12: rank=366
5,5,5: rank=98
5,5,7: rank=134
5,7,9: rank=234
5,7,10: rank=257
5,7,11: rank=280
5,8,9: rank=262
5,8,10: rank=287
5,8,11: rank=317
5,9,9: rank=296
5,9,10: rank=323
5,9,11: rank=358
5,9,12: rank=381
6,7,9: rank=270
6,7,10: rank=296
6,7,11: rank=322
6,8,10: rank=329
6,8,11: rank=365
6,9,9: rank=342
6,9,10: rank=373
6,9,11: rank=411
7,7,9: rank=318
7,7,10: rank=350
7,7,11: rank=384
7,8,9: rank=354
7,8,10: rank=393
7,8,11: rank=432
7,8,12: rank=462
7,9,9: rank=399
7,9,10: rank=441
7,9,11: rank=481
7,

# Experiment

## Floats

In [3]:
sizes_to_be_tested = [2,3,4,5,9,10,11]
random_seed_value = 2
runs = 100

alpha_results = []
naive_results = []
strassen_results = []

for trial in sizes_to_be_tested:
    print(f"Trial number {trial}")
    size = (trial,trial,trial)
    key = f"{trial},{trial},{trial}"

    alpha_standard_factor = standard_factorizations[key]
    naive_factor = generate_naive_factorization(size)
    strassen_factor = _get_2x2x2_strassen()

    n = _get_n_from_factors(alpha_standard_factor)
    
    (full_a,full_b) = _generate_random_matrices(size, random_seed_value)
    a = block_split(full_a,n,n)
    b = block_split(full_b,n,n)

    if(trial%2 != 0):
        padded_size = int(n+1)

        offsets=[0,0]
        array=full_a
        print([slice(offsets[dim], offsets[dim] + array.shape[dim]) for dim in range(array.ndim)])
        padded_array_a = pad(full_a, (padded_size,padded_size),[0,0])
        padded_array_b = pad(full_b, (padded_size,padded_size),[0,0])

        strassen_n = padded_size/(padded_size/2)
        strassen_a = block_split(padded_array_a,strassen_n,strassen_n)
        strassen_b = block_split(padded_array_b,strassen_n,strassen_n)
    else:
        strassen_n = n/(n/2)
        strassen_a = block_split(full_a,strassen_n,strassen_n)
        strassen_b = block_split(full_b,strassen_n,strassen_n)

    alpha_standard_func = algorithm_from_factors(alpha_standard_factor)
    naive_func = algorithm_from_factors(naive_factor)
    strassen_func = algorithm_from_factors(strassen_factor)


    execution_time_alpha = timeit.timeit(
        stmt='alpha_standard_func(a, b)',
        number=runs,  
        globals=globals()
    )
    alpha_avg_time = execution_time_alpha / runs
    alpha_results.append(alpha_avg_time)
    print(f"Average execution time for Alpha over {runs} runs: {alpha_avg_time} seconds")

    execution_time_naive = timeit.timeit(
        stmt='naive_func(a, b)',
        number=runs,  
        globals=globals()
    )
    naive_avg_time = execution_time_naive / runs
    naive_results.append(naive_avg_time)
    print(f"Average execution time for Naive over {runs} runs: {naive_avg_time} seconds")

    execution_time_strassen = timeit.timeit(
        stmt='strassen_func(strassen_a, strassen_b)',
        number=runs,  
        globals=globals()
    )
    strassen_avg_time = execution_time_strassen / runs
    strassen_results.append(strassen_avg_time)
    print(f"Average execution time for Strassen over {runs} runs: {strassen_avg_time} seconds")

print("Alpha Results:", alpha_results)
print("Naive Results:", naive_results)
print("Strassen Results:", strassen_results)
results_df = pd.DataFrame({
        'Trial': sizes_to_be_tested,
        'Alpha_Avg_Time': alpha_results,
        'Naive_Avg_Time': naive_results,
        'Strassen_Avg_Time': strassen_results
    })

results_df.to_csv('/Users/kev/Documents/projects/alphatensor-analysis/results/results_float.csv', index=False)

Trial number 2
Average execution time for Alpha over 100 runs: 0.00012557208006910514 seconds
Average execution time for Naive over 100 runs: 6.978042001719587e-05 seconds
Average execution time for Strassen over 100 runs: 0.00010018624998338055 seconds
Trial number 3
[slice(0, 3, None), slice(0, 3, None)]
Average execution time for Alpha over 100 runs: 0.000512431250026566 seconds
Average execution time for Naive over 100 runs: 0.0003262858399830293 seconds
Average execution time for Strassen over 100 runs: 9.285916996304878e-05 seconds
Trial number 4
Average execution time for Alpha over 100 runs: 0.001926889590031351 seconds
Average execution time for Naive over 100 runs: 0.0010957374999998137 seconds
Average execution time for Strassen over 100 runs: 0.0001089625000167871 seconds
Trial number 5
[slice(0, 5, None), slice(0, 5, None)]
Average execution time for Alpha over 100 runs: 0.0035723716599750334 seconds
Average execution time for Naive over 100 runs: 0.0029619458399974975 sec

## Ints

In [7]:
sizes_to_be_tested = [2,3,4,5,9,10,11]
random_seed_value = 2
runs = 100

alpha_results = []
naive_results = []
strassen_results = []

for trial in sizes_to_be_tested:
    print(f"Trial number {trial}")
    size = (trial,trial,trial)
    key = f"{trial},{trial},{trial}"

    alpha_standard_factor = standard_factorizations[key]
    naive_factor = generate_naive_factorization(size)
    strassen_factor = _get_2x2x2_strassen()

    n = _get_n_from_factors(alpha_standard_factor)
    
    (full_a,full_b) = _generate_random_int_matrices(size, random_seed_value)
    a = block_split(full_a,n,n)
    b = block_split(full_b,n,n)

    if(trial%2 != 0):
        padded_size = int(n+1)

        offsets=[0,0]
        array=full_a
        print([slice(offsets[dim], offsets[dim] + array.shape[dim]) for dim in range(array.ndim)])
        padded_array_a = pad(full_a, (padded_size,padded_size),[0,0])
        padded_array_b = pad(full_b, (padded_size,padded_size),[0,0])

        strassen_n = padded_size/(padded_size/2)
        strassen_a = block_split(padded_array_a,strassen_n,strassen_n)
        strassen_b = block_split(padded_array_b,strassen_n,strassen_n)
    else:
        strassen_n = n/(n/2)
        strassen_a = block_split(full_a,strassen_n,strassen_n)
        strassen_b = block_split(full_b,strassen_n,strassen_n)

    alpha_standard_func = algorithm_from_factors(alpha_standard_factor)
    naive_func = algorithm_from_factors(naive_factor)
    strassen_func = algorithm_from_factors(strassen_factor)



    execution_time_alpha = timeit.timeit(
        stmt='alpha_standard_func(a, b)',
        number=runs,  
        globals=globals()
    )
    alpha_avg_time = execution_time_alpha / runs
    alpha_results.append(alpha_avg_time)
    print(f"Average execution time for Alpha over {runs} runs: {alpha_avg_time} seconds")

    execution_time_naive = timeit.timeit(
        stmt='naive_func(a, b)',
        number=runs,  
        globals=globals()
    )
    naive_avg_time = execution_time_naive / runs
    naive_results.append(naive_avg_time)
    print(f"Average execution time for Naive over {runs} runs: {naive_avg_time} seconds")

    execution_time_strassen = timeit.timeit(
        stmt='strassen_func(strassen_a, strassen_b)',
        number=runs,  
        globals=globals()
    )
    strassen_avg_time = execution_time_strassen / runs
    strassen_results.append(strassen_avg_time)
    print(f"Average execution time for Strassen over {runs} runs: {strassen_avg_time} seconds")

print("Alpha Results:", alpha_results)
print("Naive Results:", naive_results)
print("Strassen Results:", strassen_results)
results_df = pd.DataFrame({
        'Trial': sizes_to_be_tested,
        'Alpha_Avg_Time': alpha_results,
        'Naive_Avg_Time': naive_results,
        'Strassen_Avg_Time': strassen_results
    })

results_df.to_csv('/Users/kev/Documents/projects/alphatensor-analysis/results/results_int.csv', index=False)

Trial number 2
Average execution time for Alpha over 100 runs: 0.00011043750004319008 seconds
Average execution time for Naive over 100 runs: 7.864416002121289e-05 seconds
Average execution time for Strassen over 100 runs: 9.72320800065063e-05 seconds
Trial number 3
[slice(0, 3, None), slice(0, 3, None)]
Average execution time for Alpha over 100 runs: 0.0005107824999868171 seconds
Average execution time for Naive over 100 runs: 0.00031136541998421307 seconds
Average execution time for Strassen over 100 runs: 9.181457993690855e-05 seconds
Trial number 4
Average execution time for Alpha over 100 runs: 0.0016368141699786066 seconds
Average execution time for Naive over 100 runs: 0.001008044170011999 seconds
Average execution time for Strassen over 100 runs: 0.00010058874999231193 seconds
Trial number 5
[slice(0, 5, None), slice(0, 5, None)]
Average execution time for Alpha over 100 runs: 0.00314927749997878 seconds
Average execution time for Naive over 100 runs: 0.0024868454199895498 seco